In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from google.colab import files

In [ ]:
files.upload()

In [4]:
spam = pd.read_csv("spam.csv")
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])



In [9]:
mensagens = spam['Message'].values
X_train, X_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)


In [10]:
token = Tokenizer(num_words=1000)
token.fit_on_texts(X_train)

X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)


In [11]:
X_train = pad_sequences(X_train, padding="post", maxlen=500)
X_test = pad_sequences(X_test, padding="post", maxlen=500)


In [12]:
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=50, input_length=500))
modelo.add(Flatten())

modelo.add(Dense(units=8, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation="sigmoid"))

In [13]:
modelo.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           372700    
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 8)                 200008    
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 572,717
Trainable params: 572,717
Non-trainable params: 0
_________________________________________________________________


In [14]:
modelo.fit(X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test))

Epoch 1/20
390/390 [==============================] - 4s 9ms/step - loss: 0.0577 - accuracy: 0.9274 - val_loss: 0.0214 - val_accuracy: 0.9755
Epoch 2/20
390/390 [==============================] - 3s 8ms/step - loss: 0.0150 - accuracy: 0.9846 - val_loss: 0.0137 - val_accuracy: 0.9833
Epoch 3/20
390/390 [==============================] - 3s 8ms/step - loss: 0.0102 - accuracy: 0.9910 - val_loss: 0.0120 - val_accuracy: 0.9862
Epoch 4/20
390/390 [==============================] - 3s 8ms/step - loss: 0.0091 - accuracy: 0.9933 - val_loss: 0.0126 - val_accuracy: 0.9868
Epoch 5/20
390/390 [==============================] - 3s 8ms/step - loss: 0.0079 - accuracy: 0.9946 - val_loss: 0.0124 - val_accuracy: 0.9862
Epoch 6/20
390/390 [==============================] - 3s 8ms/step - loss: 0.0067 - accuracy: 0.9951 - val_loss: 0.0127 - val_accuracy: 0.9862
Epoch 7/20
390/390 [==============================] - 3s 9ms/step - loss: 0.0061 - accuracy: 0.9951 - val_loss: 0.0129 - val_accuracy: 0.9862
Epoch 

In [15]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print("Loss: ", loss)
print("Acurácia: ", accuracy)

53/53 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.9833
Loss:  0.015393597073853016
Acurácia:  0.9832535982131958


In [16]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

53/53 [==============================] - 0s 2ms/step
[[8.4158834e-12]
 [9.9999213e-01]
 [9.9999923e-01]
 ...
 [1.4845166e-13]
 [7.3769003e-12]
 [3.2679785e-07]]


In [17]:
prev = (nova_previsao > 0.5)
print(prev)

[[False]
 [ True]
 [ True]
 ...
 [False]
 [False]
 [False]]


In [18]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[1450    2]
 [  26  194]]
